In [207]:
import pandas as pd
import numpy as np
import re

def paranthesis_remover(entry):
    return re.sub('\(.+?\)', '', str(entry))

def one_hot_encoder(file_name):
    imdb_budget_df = pd.read_csv(file_name)
    
    # Include only movies, and successful API calls
    reduced_imdb = imdb_budget_df.loc[imdb_budget_df['Response'] == True]
    reduced_imdb = imdb_budget_df.loc[imdb_budget_df['Type'] == 'movie']
    
    # drop extra columns
    reduced_imdb.columns = map(str.lower, reduced_imdb.columns)
    return reduced_imdb.drop(columns=['actors', 'director', 'genre', 'language', 'country', 'dvd', 'writer', 'unnamed: 0', 'awards',
                                      'plot', 'error', 'poster', 'production',
                                      'rated', 'ratings', 'released', 'response', 'runtime',
                                      'type', 'website', 'imdbid', 'totalseasons'])
    

In [208]:
imdb = one_hot_encoder('raw_data/combined_imdb_data.csv')

In [209]:
def preprocess(df):
    df['boxoffice'] = df['boxoffice'].apply(lambda x: ''.join([ch for ch in x if ch.isdigit()]))
    df['imdbvotes'] = df['imdbvotes'].apply(lambda x: ''.join([ch for ch in x if ch.isdigit()]))
    return df

budget_df = imdb.dropna(subset=['boxoffice'])
budget_df = preprocess(budget_df)

C:\Users\Harjee\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Harjee\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [210]:
oscar_data = pd.read_csv('./processed_data/oscar_data.csv').rename(columns={"entity": "title", "category": "nominated", "winner": "won"})

In [211]:
budget_df = pd.merge(budget_df, oscar_data, how='inner', on='title')


In [212]:
def grab_first(entry):
    return entry.split(',')[0]

#budget_df['country'] = budget_df['country'].apply(grab_first)

In [213]:
"""import matplotlib.pyplot as plt

foid = foreign_oscar_imdb_df[['country', 'won', 'year_x']]
foid.sort_values('year_x', inplace=True)
foid = foid[foid['won'] == True]

winning_countries = foid.groupby(['country']).sum()
winning_countries = winning_countries.reset_index().sort_values('won', ascending=False)
winning_countries = winning_countries.nlargest(10, columns='won')
winning_countries.plot(x='country', y='won', kind='bar', rot=0, figsize=(12, 8), title='Foreign Oscar Winners By Country')"""

"import matplotlib.pyplot as plt\n\nfoid = foreign_oscar_imdb_df[['country', 'won', 'year_x']]\nfoid.sort_values('year_x', inplace=True)\nfoid = foid[foid['won'] == True]\n\nwinning_countries = foid.groupby(['country']).sum()\nwinning_countries = winning_countries.reset_index().sort_values('won', ascending=False)\nwinning_countries = winning_countries.nlargest(10, columns='won')\nwinning_countries.plot(x='country', y='won', kind='bar', rot=0, figsize=(12, 8), title='Foreign Oscar Winners By Country')"

In [214]:
"""def grab_first_two(entry):
    return ','.join(sorted(entry.split(',')[0:1]))
    
budget_df['genre'] = budget_df['genre'].apply(grab_first_two)
budget_df['director'] = budget_df['director'].apply(grab_first)
budget_df['language'] = budget_df['language'].apply(grab_first)"""

"def grab_first_two(entry):\n    return ','.join(sorted(entry.split(',')[0:1]))\n    \nbudget_df['genre'] = budget_df['genre'].apply(grab_first_two)\nbudget_df['director'] = budget_df['director'].apply(grab_first)\nbudget_df['language'] = budget_df['language'].apply(grab_first)"

In [215]:
"""genre_df = budget_df[['genre', 'won', 'year_x']]
genre_df.sort_values('year_x', inplace=True)
genre_df = genre_df[genre_df['won'] == True]

winning_genres = genre_df.groupby(['genre']).sum()
winning_genres = winning_genres.reset_index().sort_values('won', ascending=False)
winning_genres = winning_genres.nlargest(10, columns='won')
winning_genres.plot(x='genre', y='won', kind='bar', rot=0, figsize=(12, 8), title='Oscar Winners By Primary Genre')"""

"genre_df = budget_df[['genre', 'won', 'year_x']]\ngenre_df.sort_values('year_x', inplace=True)\ngenre_df = genre_df[genre_df['won'] == True]\n\nwinning_genres = genre_df.groupby(['genre']).sum()\nwinning_genres = winning_genres.reset_index().sort_values('won', ascending=False)\nwinning_genres = winning_genres.nlargest(10, columns='won')\nwinning_genres.plot(x='genre', y='won', kind='bar', rot=0, figsize=(12, 8), title='Oscar Winners By Primary Genre')"

In [223]:
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

features = budget_df.drop(columns=['won', 'title', 'nominated'])
labels = budget_df['won']

"""
X_train, X_test, y_train, y_test = train_test_split(features, labels, random_state=69)

pca = PCA(n_components=2)
X_train_pca = pca.fit_transform(X_train)"""

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').